# Lab correction : Spark Introduction

## Hand-on 1 : data importatation

In [1]:
df_tweet_small = spark.read.json("s3://mon-super-bucket-06042021/tweets/tweets20210414-143312.jsonl.gz")
df_tweet_big = spark.read.json("s3://mon-super-bucket-06042021/tweets/")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1618489970146_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Concern about the importation : curently the data are not really in the cluster. Those two object do not contain any data. Each time we run a computation, Spark download the data. To avoid this, we need to cache those df

In [5]:
df_tweet_small.cache()
df_tweet_big.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[auteur: string, contenu: string, date_creation: string, entities: struct<annotations:array<struct<end:bigint,normalized_text:string,probability:double,start:bigint,type:string>>,cashtags:array<struct<end:bigint,start:bigint,tag:string>>,hashtags:array<struct<end:bigint,start:bigint,tag:string>>,mentions:array<struct<end:bigint,start:bigint,username:string>>,urls:array<struct<description:string,display_url:string,end:bigint,expanded_url:string,images:array<struct<height:bigint,url:string,width:bigint>>,start:bigint,status:bigint,title:string,unwound_url:string,url:string>>>, hashtags: array<string>, like_count: bigint, other: struct<auteur_name:string>, reply_count: bigint, retweet_count: bigint]

## Hand-on 2 : DataFrame basic manipulations

- Define a dataframe `tweet_author_hashtags`  with only the `auteur` and `hashtags` columns

In [6]:
tweet_author_hashtags = df_tweet_big.select("auteur", "hashtags")
tweet_author_hashtags.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------------+
|      auteur|            hashtags|
+------------+--------------------+
|    rewbirth|                  []|
|  CO2Flandre|    [モンハンライズ]|
|    hnkn_nrm|                  []|
|    yushuxn_|                  []|
|AsuranManivs|[HBDVijayaLakshmi...|
+------------+--------------------+
only showing top 5 rows

- Print a dataframe with only the `auteur`, `mentions`, and `cashtags` columns. `mentions`, and `cashtags` are both nested columns in `entities`

In [7]:
df_tweet_big.select("auteur", "entities.cashtags", "entities.mentions").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------+--------------------+
|      auteur|cashtags|            mentions|
+------------+--------+--------------------+
|    rewbirth|    null|  [[11, 3, h7unsik]]|
|  CO2Flandre|    null|[[15, 3, Jin_No_U...|
|    hnkn_nrm|    null|  [[11, 3, wool103]]|
|    yushuxn_|    null|[[16, 3, ugly_sun...|
|AsuranManivs|    null|[[13, 3, DhanushF...|
+------------+--------+--------------------+
only showing top 5 rows

- Filter your first dataframe and only keep tweets with more than 1 like . Save this new dataframe and print it.

In [8]:
more_than_one_like = df_tweet_big.filter(df_tweet_big.like_count > 0).select("auteur", "like_count")
more_than_one_like.show(5)
#The print are prett

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+----------+
|         auteur|like_count|
+---------------+----------+
|  signal_frqncy|         1|
|        ripo_FN|         1|
|    nazuna__125|         1|
|TpaGeILXCCd7cd4|         1|
|  piscesrihanna|         1|
+---------------+----------+
only showing top 5 rows

## Hand-on 3 : Basic DataFrame column manipulation 

- Define a dataframe with a column names `interaction_count`. This column is the sum of like_count, reply_count and retweet_count.

In [9]:
df_tweet_big_interation = df_tweet_big.withColumn("interaction_count", df_tweet_big.like_count+df_tweet_big.reply_count+df_tweet_big.retweet_count )
df_tweet_big_interation.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------------+--------------------+-----------------------+--------------------+----------+---------------+-----------+-------------+-----------------+
|      auteur|             contenu|       date_creation|               entities|            hashtags|like_count|          other|reply_count|retweet_count|interaction_count|
+------------+--------------------+--------------------+-----------------------+--------------------+----------+---------------+-----------+-------------+-----------------+
|    rewbirth|RT @h7unsik: btob...|2021-04-14T12:33:...|   [[[16, btob, 0.83...|                  []|         0|         [erwy]|          0|            9|                9|
|  CO2Flandre|RT @Jin_No_Uchi: ...|2021-04-14T12:33:...|[,, [[38, 30, モンハ...|    [モンハンライズ]|         0|          [CO2]|          0|        11646|            11646|
|    hnkn_nrm|RT @wool103: http...|2021-04-14T12:33:...|   [,,, [[11, 3, woo...|                  []|         0|       [のらめ]|          0|        

In [10]:
df_tweet_big = df_tweet_big.drop("other")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Hand-on 4: Advance DataFrame column manipulation - Array manipulation

- Filter all the tweets without hashtags and for each remained lines, split its hashtag to only get one hashtag by line. 


In [11]:
from pyspark.sql.functions import explode, size
df_tweet_big.filter(size(df_tweet_big.hashtags) > 0).withColumn("hashtag", explode(df_tweet_big.hashtags)).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+------------------------------------+--------------------+-----------------------+--------------------+----------+-----------+-------------+--------------------+
|        auteur|                             contenu|       date_creation|               entities|            hashtags|like_count|reply_count|retweet_count|             hashtag|
+--------------+------------------------------------+--------------------+-----------------------+--------------------+----------+-----------+-------------+--------------------+
|    CO2Flandre|                RT @Jin_No_Uchi: ...|2021-04-14T12:33:...|[,, [[38, 30, モンハ...|    [モンハンライズ]|         0|          0|        11646|      モンハンライズ|
|  AsuranManivs|                RT @DhanushFc: பி...|2021-04-14T12:33:...|   [,, [[96, 75, HBD...|[HBDVijayaLakshmi...|         0|          0|           56|HBDVijayaLakshmiAmma|
|  AsuranManivs|                RT @DhanushFc: பி...|2021-04-14T12:33:...|   [,, [[96, 75, HBD...|[HBDVijayaLakshmi...|        

- Create a new column with the number of words of the `contenu` column (split + size)

In [12]:
from pyspark.sql.functions import split
number_of_word = df_tweet_big.withColumn("number_word", size(split("contenu", " ")))
number_of_word.select('number_word').show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+
|number_word|
+-----------+
|          9|
|          4|
|          4|
|          6|
|          8|
+-----------+
only showing top 5 rows

- Count how many tweets contain the COVID19 hashtag.

In [13]:
from pyspark.sql.functions import array_contains
df_tweet_big.filter(array_contains("hashtags", "COVID19")).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2

### Hands-on 5 : Advance DataFrame column manipulation - User defined function
- Create an udf that count how many words a tweet contains.

In [14]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

def word_count(string):
    return len(string.split(" "))

word_count_udf = udf(lambda x : word_count(x), IntegerType())

df_tweet_big.withColumn("word_count", word_count_udf("contenu")).select("word_count").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|word_count|
+----------+
|         9|
|         4|
|         4|
|         6|
|         8|
+----------+
only showing top 5 rows

## Hand-on 6 : Aggregation functions

- What are the min, max, average of `interaction_count` ?

In [15]:
from pyspark.sql.functions import min, max, avg, first, last, sum, sumDistinct
df_tweet_big_interation.select(min("interaction_count"), max("interaction_count"), avg("interaction_count")).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------+----------------------+----------------------+
|min(interaction_count)|max(interaction_count)|avg(interaction_count)|
+----------------------+----------------------+----------------------+
|                     0|                285715|              1718.679|
+----------------------+----------------------+----------------------+

- How many hashtags do we have ? Distinct hashtags ? Try the approximative count with 0.1, 0.01 and 0.001 as maximum estimation error allowed.

In [16]:
from pyspark.sql.functions import count
df_tweet_big.select(count("hashtags")).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+
|count(hashtags)|
+---------------+
|           3000|
+---------------+

In [17]:
from pyspark.sql.functions import countDistinct
df_tweet_big.select(countDistinct("hashtags")).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------+
|count(DISTINCT hashtags)|
+------------------------+
|                     412|
+------------------------+

In [18]:
from pyspark.sql.functions import approx_count_distinct
df_tweet_big.select(approx_count_distinct("hashtags", 0.1)).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------------+
|approx_count_distinct(hashtags)|
+-------------------------------+
|                            421|
+-------------------------------+

In [19]:
df_tweet_big.select(approx_count_distinct("hashtags", 0.01)).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------------+
|approx_count_distinct(hashtags)|
+-------------------------------+
|                            413|
+-------------------------------+

In [6]:
# df_tweet_big.select(approx_count_distinct("hashtags", 0.001)).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

KeyboardInterrupt: 

## Hand-on 7 : Grouping functions
- Compute a daframe with the min, max and average retweet of each `auteur`. Then sort it (using the `sort(column : string)` method) and print it.

In [25]:
from pyspark.sql.functions import desc
df_tweet_big.groupBy("auteur").agg(min("retweet_count").alias("min_rt"), max("retweet_count").alias("max_rt"), avg("retweet_count").alias("avg_rt")).orderBy(desc("max_rt")).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+------+------+--------+
|       auteur|min_rt|max_rt|  avg_rt|
+-------------+------+------+--------+
| liliaballard|285715|285715|285715.0|
|   xthelilone|257913|257913|257913.0|
| exopiteopaen| 76917| 76917| 76917.0|
|   realllllpw| 76882| 76882| 76882.0|
|Sukunya812Chp| 71479| 71479| 71479.0|
+-------------+------+------+--------+
only showing top 5 rows